## **MIDI Factory**

#### Import Libraries

In [292]:
import pandas as pd
import xlwings as xw
import math

from miditoolkit.midi import parser as mid_parser  
from miditoolkit.midi import containers as ct
from miditoolkit import MidiFile

### load midi file

In [293]:
path_midi = 'raw.mid'

### Constant

In [294]:
ticks_per_beat = round(midi_obj.ticks_per_beat / 10)
time_signature = midi_obj.time_signature_changes[0]
numerator =  time_signature.numerator
_denominator =  time_signature.denominator
tempo = round(midi_obj.tempo_changes[0].tempo)
restNote = None

rhythms = {
    round(ticks_per_beat * 4): "w", 
    round(ticks_per_beat * 3): "hd",
    round(ticks_per_beat * 4 / 3): "ht",
    round(ticks_per_beat * 2): "h",
    round(ticks_per_beat * 1.5): "qd",
    round(ticks_per_beat * 2 / 3): "qt",
    round(ticks_per_beat * 1 + ticks_per_beat * 0.25): "q^s",
    ticks_per_beat: "q",
    round(ticks_per_beat * 1 / 3):"et", 
    round(ticks_per_beat * 3 / 4): "ed",
    round(ticks_per_beat / 2): "e",
    round(ticks_per_beat / 4 ): "s",
    round(ticks_per_beat / 6): "st"
}

theme_start = "theme_start"
phrase_start = "phrase_start"

bar_duration = numerator * ticks_per_beat

print('✥ ticks per beat:', ticks_per_beat)
print('✥ bar duration:', bar_duration)
print('✥ tempo:', tempo, "BPM")

✥ ticks per beat: 10
✥ bar duration: 40
✥ tempo: 200 BPM


### Define RHYTHMS

In [295]:
def rhythms_define(duration: int):
    return duration
    for i in range(-2, 3):
        rhythm = rhythms.get(duration + i)
        if rhythm:
            return rhythm

### Get Markers

In [296]:
def markers_set(listOfMarkers):
    markers_set = set()
    for marker in listOfMarkers:
        marker_titles = marker.text.split(',')
        for m in marker_titles:
            markers_set.add(m)
    return markers_set

def markers_dict(listOfMarkers):
    markers = {}
    for marker in listOfMarkers:
        marker_titles = marker.text.split(',')
        markers[round(marker.time / 10)] = marker_titles
    return markers


### Get Pitch bend

In [297]:
def get_quarter_note(midi_obj: MidiFile):
    pitch_bends_raw = midi_obj.instruments[0].pitch_bends

    pitch_bends_filtered = list(filter(lambda item: item.pitch != 0, pitch_bends_raw))

    pitch_bends = {}

    for pitch_bend in pitch_bends_filtered:
        pitch_bends[round(pitch_bend.time / 10)] = pitch_bend.pitch
        
    return pitch_bends

### Create DataFrame with empty Series

In [298]:
def midi_df(markers_set: set):
    # Create empty Series
    empty_series = pd.Series()
    
    # DataFrame basics
    midi_df = pd.DataFrame({
        'note': empty_series,
        'interval': empty_series,
        'note_start_time': empty_series,
        'duration': empty_series,
        'slur': empty_series,
        'velocity': empty_series,
    })
    
    # add markers columns
    for column in markers_set:
        midi_df[column] = empty_series
        
    return midi_df

### Create New Row

In [299]:
def new_entry(start_time: None | int, duration: None | int, Def_column, markers, note= restNote, velocity= None, slur: int= 0, last_note= None, last_note_end_time= 0 ):
    row = {key: 0 for key in Def_column}
    row['note'] = note if(note and velocity) else restNote
    row['note_start_time'] = start_time
    row['duration'] = rhythms_define(duration)
    row['velocity'] = velocity
    row['interval'] = None
            
    if(markers):
        for marker in markers:
            row[marker] = 1
            
    if(slur):
        row['slur'] = slur
        
    
    if(note != None  and velocity):
        if(row.get(theme_start) == 0 and row.get(phrase_start) == 0):
            last_note_int = last_note or 0
            current_note = row['note'] or 0
            interval = current_note - last_note_int
            row['interval'] =  interval if(start_time - last_note_end_time < bar_duration) else None
        else:
            row['interval'] = None
            
    return row

### Load your MIDI file

In [300]:
def midi_to_dataFrame(midi_obj: MidiFile ):
    notes = midi_obj.instruments[0].notes
    quarter_tone_list = get_quarter_note(midi_obj)
    midi_dataFrame= midi_df(markers_set(midi_obj.markers))
    markers = markers_dict(midi_obj.markers)
    columns = midi_dataFrame.columns
    
    last_note: None | int  = None
    last_note_end_time = 0
    quarter_tone = False

    for note in notes:
        pitch = note.pitch
        start_time = round(note.start / 10)
        end_time = round(note.end / 10)
        pitch_bend = quarter_tone_list.get(start_time)
        quarter_tone = pitch_bend / 341 * 0.5 if(pitch_bend) else 0
        rest_exist = start_time - last_note_end_time > 0
        
        # handle rest if exist
        if (rest_exist):
            rest_start_time = last_note_end_time
            next_note_start_time = start_time
            
            rest_starts_bar = math.ceil((rest_start_time + 1) / bar_duration)
            rest_ends_bar = math.ceil((next_note_start_time - 1) / bar_duration)

            existed_rest_bars = rest_ends_bar - rest_starts_bar
            
            
            for i in range(existed_rest_bars):
                bar_ends = (rest_starts_bar + i) * bar_duration
                
                # Add a new rest event to the DataFrame  
                midi_dataFrame.loc[len(midi_dataFrame)] = new_entry(
                    start_time= rest_start_time,
                    duration= bar_ends - rest_start_time,
                    Def_column= columns,
                    markers= markers.get(rest_start_time)
                )
                
                # update new rest start point
                rest_start_time = bar_ends
                
            # existed_rest_less_than_bar 
            if(next_note_start_time - rest_start_time):
                # Add a new rest event to the DataFrame
                midi_dataFrame.loc[len(midi_dataFrame)] = new_entry(
                    start_time= rest_start_time,
                    duration= start_time - rest_start_time,
                    Def_column= columns,
                    markers= markers.get(rest_start_time)
                )

        
        note_starts_bar = math.ceil((start_time + 1) / bar_duration)
        note_ends_bar = math.ceil((end_time - 1) / bar_duration)
        nested_slur_note = False

        existed_slur_note = note_ends_bar - note_starts_bar
        ## handle slur notes
        if(existed_slur_note):
            for i in range(existed_slur_note):
                slur = 0
                if(nested_slur_note):
                    slur = 1
                else:
                    nested_slur_note = not nested_slur_note
                
                bar_ends = (note_starts_bar + i) * bar_duration
                current_note = pitch + quarter_tone
                
                # Add a new note event to the DataFrame  
                midi_dataFrame.loc[len(midi_dataFrame)] = new_entry(
                    note= current_note,
                    start_time= start_time,
                    duration= bar_ends - start_time,
                    velocity= note.velocity,
                    slur= slur,
                    Def_column= columns,
                    last_note= last_note,
                    last_note_end_time= last_note_end_time,
                    markers= markers.get(start_time)
                )
                
                # update new note start point
                last_note_end_time = bar_ends
                start_time = bar_ends
                last_note = current_note
                
            bar_ends = (note_starts_bar + i) * bar_duration
            current_note = pitch + quarter_tone
            
            ## there is nested slur note and it duration is less than bar duration
            if(end_time % bar_duration):
                # Add a new note event to the DataFrame  
                midi_dataFrame.loc[len(midi_dataFrame)] = new_entry(
                    note= current_note,
                    start_time= start_time,
                    duration= end_time - start_time,
                    velocity= note.velocity,
                    slur= 1,
                    Def_column= columns,
                    last_note= last_note,
                    last_note_end_time= last_note_end_time,
                    markers= markers.get(start_time)
                )
            # update new note start point
            last_note_end_time = end_time
            last_note = current_note
        ## handle none-slur notes
        else:
            bar_ends = (note_starts_bar + i) * bar_duration
            current_note = pitch + quarter_tone
            
            # Add a new note event to the DataFrame
            midi_dataFrame.loc[len(midi_dataFrame)] = new_entry(
                note= current_note,
                start_time= start_time,
                duration= end_time - start_time,
                velocity= note.velocity,
                Def_column= columns,
                last_note= last_note,
                last_note_end_time= last_note_end_time,
                markers= markers.get(start_time)
            )
            
            # update new note start point
            last_note_end_time = end_time
            last_note = current_note
    
    return midi_dataFrame

In [301]:

dataFrame = midi_to_dataFrame(
    midi_obj= mid_parser.MidiFile(f'./midi/{path_midi}')
)

### Update Excel File

In [302]:
# Connect to the workbook
workbook = xw.Book('test.xlsx')

# Select the worksheet
worksheet = workbook.sheets[path_midi.split('.')[0]]

# Delete the existing table (if it exists)
if worksheet.tables:
    worksheet.range(worksheet.tables[0].range.address).api.Delete()

# Add a new table to the worksheet
table = worksheet.tables.add(source=worksheet.range('A1'), name='MyTable')

# Update the table with the new data
table.update(dataFrame)

# Show the updated range of cells in a new window
table.range.api.Select()

True